<a href="https://colab.research.google.com/github/muke888/Personal-Projects/blob/master/RegoBot/AusRegoBot_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VERSION 2

In [1]:
#! cat /etc/os-release
!pip install selenium -q
!apt-get update -q # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver -q
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('prefs', {'profile.managed_default_content_settings.images': 2}) # Avoid loading images
chrome_options.add_experimental_option('prefs', {'disk-cache-size': 2000000000}) #4096
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--window-size=1920,1080')
#chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--disable-gpu')
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import re, time
from bs4 import BeautifulSoup
from collections import defaultdict

     |████████████████████████████████| 911kB 2.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu

In [2]:
def WA(rego):
  st=time.time()
  with webdriver.Chrome('chromedriver',options=chrome_options) as wd:
    wd.get('https://online.transport.wa.gov.au/webExternal/registration/')
    current_url = wd.current_url
    element = wd.find_element_by_xpath('/html/body/span/div/div[3]/div/div/form/ol[1]/li[1]/div/input')
    element.send_keys(str(rego))
    wd.find_element_by_xpath('/html/body/span/div/div[3]/div/div/form/ol[2]/li/div/input').click()
    time.sleep(4)
    try:
      result = wd.find_element_by_css_selector('#content > div > div.licensing-big-form > table > tbody > tr:nth-child(2) > td.data > span').text
      print(result, round(time.time()-st,2))
    except:
      print('no rego found', round(time.time()-st,2))
      wd.save_screenshot('screenie1.png')


In [3]:
def QLD(rego):
  st=time.time()
  with webdriver.Chrome('chromedriver',options=chrome_options) as wd:
    wd.get('https://www.service.transport.qld.gov.au/checkrego/application/TermAndConditions.xhtml')
    current_url = wd.current_url
    wd.find_element_by_id('tAndCForm:confirmButton').click()
    WebDriverWait(wd, 5).until(EC.url_changes(current_url))
    element = wd.find_element_by_id('vehicleSearchForm:plateNumber')
    element.send_keys(rego)
    wd.find_element_by_id('vehicleSearchForm:confirmButton').click()  
    try:
      WebDriverWait(wd, 5).until(EC.url_changes(current_url))
      result = wd.find_element_by_css_selector('#j_id_2d > dl:nth-child(2) > dd:nth-child(4)').text
      print(result, round(time.time()-st,2))
      #data['QLD'] = result, round(time.time()-st,2)
    except:
      print('no rego found', round(time.time()-st,2))
      #wd.save_screenshot('screenie4.png')


In [4]:
def ACT(rego):
  st=time.time()
  with webdriver.Chrome('chromedriver',options=chrome_options) as wd:
    wd.get('https://rego.act.gov.au/regosoawicket/public/reg/FindRegistrationPage')
    #current_url = wd.current_url
    element = wd.find_element_by_id('plateNumber')
    element.send_keys(rego)
    element.send_keys(Keys.RETURN)
    try:
      #WebDriverWait(wd, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#idb > div > div > div > div > div > table > tbody > tr > td:nth-child(1) > a > span')))
      time.sleep(2)
      #WebDriverWait(wd, 10)#.until(EC.url_changes(current_url))
      #wd.save_screenshot('screenie2.png')
      current_url = wd.current_url
      wd.find_element_by_css_selector('#id1 > div > div > div > div > div > table > tbody > tr > td:nth-child(1) > a').click()

      WebDriverWait(wd, 5).until(EC.url_changes(current_url))
      result = wd.find_element_by_css_selector('#expiryDate')
      print(result.get_attribute('value'), round(time.time()-st,2))
      #wd.save_screenshot('screenie3.png')
    except:
      print('no rego found', round(time.time()-st,2))
      #wd.save_screenshot('screenie4.png')

In [5]:
def SA(rego):
  st=time.time()
  with webdriver.Chrome('chromedriver',options=chrome_options) as wd:
    wd.get('https://account.ezyreg.sa.gov.au/account/check-registration.htm')
    WebDriverWait(wd, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#plateNumber')))
    #current_url = wd.current_url
    element = wd.find_element_by_xpath("//input[@id='plateNumber']")
    element.send_keys(rego)
    element.send_keys(Keys.RETURN)
    try:
      WebDriverWait(wd, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#registration-details-single > div:nth-child(1) > div.col-sm-6.col-xs-6.text-left > div')))
      result = wd.find_element_by_css_selector('#registration-details-single > div:nth-child(1) > div.col-sm-6.col-xs-6.text-left > div')
      print(result.get_attribute("textContent"), round(time.time()-st,2))
    except:
      print('no rego found', round(time.time()-st,2))
      wd.save_screenshot('screenie4.png')

In [6]:
def nsw(rego):
  st=time.time()
  with webdriver.Chrome('chromedriver',options=chrome_options) as wd:
    wd.get('https://my.service.nsw.gov.au/MyServiceNSW/index#/rms/freeRegoCheck/details')
    WebDriverWait(wd, 15).until(EC.presence_of_element_located((By.XPATH, './/*[contains(@id, "_input_plateNumber_")]')))
    element = wd.find_element_by_xpath('.//*[contains(@id, "_input_plateNumber_")]')
    
    wd.find_element_by_id('formly_2_checkbox-label-with-action_termsAndConditions_1').click() #Checkbox ticked
    element.send_keys(rego)
    current_url = wd.current_url
    element.submit()
    #wd.find_element_by_xpath('/html/body/div[3]/div/div/div/div/div/div[2]/free-rego-check-details/form/div[3]/button').click()
    try:
      WebDriverWait(wd, 5).until(EC.url_changes(current_url))
      #wd.save_screenshot('screenie5.png')
      result = wd.find_element_by_css_selector('body > div.container > div > div > div > div > div > div.ng-scope > free-rego-check-display > div:nth-child(3) > div > div:nth-child(6) > div:nth-child(2) > div:nth-child(4) > strong').text
      print(result, round(time.time()-st,2)) #result[-3]
      data['nsw'] = result, round(time.time()-st,2)
    except:
      print ('no rego found', round(time.time()-st,2))
      #wd.save_screenshot('screenie5.png')
      #data['nsw'] = "no rego found", round(time.time()-st,2)



In [7]:
def vic(rego):
  st=time.time()
  with webdriver.Chrome('chromedriver',options=chrome_options) as wd:
    wd.get('https://www.vicroads.vic.gov.au/registration/buy-sell-or-transfer-a-vehicle/check-vehicle-registration/vehicle-registration-enquiry')

    element = wd.find_element_by_xpath('.//*[contains(@id, "RegistrationNumber_CtrlHolderDivShown")]') #if ID contains
    element.send_keys(rego)
    element = wd.find_element_by_xpath('.//*[contains(@id, "panel_btnSearch")]').click()
    #time.sleep(1)
    src = str(BeautifulSoup(wd.page_source).find_all(attrs={'class': 'detail-module'}))
    start= ' expiry date:</label>\n<div class="display"> '
    end = '</div>\n</div>\n<div'
    try:
      result = re.search('%s(.*)%s' % (start, end), src).group(1)
      print(result, round(time.time()-st,2))
      data['vic'] = result, round(time.time()-st,2)
    except:
      print ('no rego found', round(time.time()-st,2))
      #data['vic'] = "no rego found", round(time.time()-st,2)
    #model = re.search('%s(.*)%s' % (start2, end), src).group(1)
    #VIN = re.search('%s(.*)%s' % (start3, end), src).group(1)
    #engine = re.search('%s(.*)%s' % (start4, end), src).group(1)

In [8]:
data = {"vic": None, "nsw": None, "SA": None}
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait, as_completed
st = time.time()
exec_list = []

with ThreadPoolExecutor(max_workers=4) as executor:
    future1=executor.submit(vic, 'ARR655')
    future2=executor.submit(nsw, 'ARR655')
    future3=executor.submit(SA, 'ARR655')
    #executors_list.append(executor.submit(my_func3, arg1, arg2))

    while True:
      if any(v is not None for v in data.values()):
        executor._threads.clear()
        concurrent.futures.thread._threads_queues.clear()
        break
        #sys.exit()

print(data ,round(time.time()-st,2))

no rego found 12.47
Current - 11/12/2020 14.71
{'vic': ('Current - 11/12/2020', 14.71), 'nsw': None, 'SA': None} 14.72
